In [ ]:
# --- Etapa 1: Instalação das Bibliotecas ---
# Este comando instala a biblioteca ortools do Google, essencial para resolver o problema de otimização.
!pip install ortools

# --- Etapa 2: Definição das Estruturas de Dados (Classes) ---
# Definimos as classes que representarão os componentes do nosso problema: Professor, Disciplina, Sala e Turma.

class Professor:
    """Representa um professor, com seu nome e as disciplinas que ele pode lecionar."""
    def __init__(self, nome, disciplinas_que_leciona):
        self.nome = nome
        self.disciplinas_que_leciona = disciplinas_que_leciona

class Disciplina:
    """Representa uma disciplina, com seu nome e a necessidade (ou não) de um laboratório."""
    def __init__(self, nome, requer_laboratorio):
        self.nome = nome
        self.requer_laboratorio = requer_laboratorio

class Sala:
    """Representa uma sala de aula, com seu nome, capacidade de alunos e se é um laboratório."""
    def __init__(self, nome, capacidade, eh_laboratorio):
        self.nome = nome
        self.capacidade = capacidade
        self.eh_laboratorio = eh_laboratorio

class Turma:
    """Representa uma turma de alunos, com seu nome, quantidade de alunos e as disciplinas que cursa."""
    def __init__(self, nome, numero_de_alunos, disciplinas):
        self.nome = nome
        self.numero_de_alunos = numero_de_alunos
        self.disciplinas = disciplinas

# --- Etapa 3: Criação dos Dados de Exemplo ---
# Aqui, criamos as instâncias das classes que serão usadas para resolver o problema.

print("1. Criando os dados de exemplo...")

# Disciplinas
matematica = Disciplina("Matemática", requer_laboratorio=False)
portugues = Disciplina("Português", requer_laboratorio=False)
quimica = Disciplina("Química", requer_laboratorio=True)
fisica = Disciplina("Física", requer_laboratorio=False)
historia = Disciplina("História", requer_laboratorio=False)
disciplinas = [matematica, portugues, quimica, fisica, historia]

# Professores
joao = Professor("João", [matematica, fisica])
maria = Professor("Maria", [portugues, historia])
pedro = Professor("Pedro", [quimica])
professores = [joao, maria, pedro]

# Salas
sala_101 = Sala("Sala 101", capacidade=30, eh_laboratorio=False)
sala_102 = Sala("Sala 102", capacidade=40, eh_laboratorio=False)
laboratorio_quimica = Sala("Laboratório de Química", capacidade=25, eh_laboratorio=True)
salas = [sala_101, sala_102, laboratorio_quimica]

# Turmas
turma_101 = Turma("Turma 101", numero_de_alunos=25, disciplinas=[matematica, portugues, quimica])
turma_202 = Turma("Turma 202", numero_de_alunos=35, disciplinas=[fisica, historia, matematica])
turmas = [turma_101, turma_202]

# Horários
horarios = ["Segunda_08h", "Segunda_09h", "Terca_08h", "Terca_09h", "Quarta_08h", "Quarta_09h"]

print("Dados de exemplo criados com sucesso!")

# --- Etapa 4: Configuração do Modelo de Otimização ---

print("\n2. Configurando o modelo de otimização...")

# Importação de bibliotecas necessárias
from ortools.sat.python import cp_model
import matplotlib.pyplot as plt

# Inicializa o modelo
model = cp_model.CpModel()

# Cria um dicionário para mapear qual professor leciona qual disciplina
professores_por_disciplina = {d.nome: [] for d in disciplinas}
for p in professores:
    for d in p.disciplinas_que_leciona:
        professores_por_disciplina[d.nome].append(p)

# Cria as variáveis de decisão
# Cada variável `aulas_vars` será booleana (0 ou 1), indicando se uma aula acontece em um local e horário específicos.
aulas_vars = {}
for turma in turmas:
    for disciplina in turma.disciplinas:
        for sala in salas:
            for horario in horarios:
                var_name = f"{turma.nome}_{disciplina.nome}_{sala.nome}_{horario}"
                aulas_vars[(turma, disciplina, sala, horario)] = model.NewBoolVar(var_name)

print("Modelo e variáveis de decisão criados.")

# --- Etapa 5: Adição das Restrições Rígidas ---
# Restrições são as regras que o modelo DEVE seguir para gerar uma solução válida.

print("\n3. Adicionando as restrições rígidas...")

# Restrição 1: Cada aula (disciplina de uma turma) deve ser alocada exatamente uma vez.
for turma in turmas:
    for disciplina in turma.disciplinas:
        model.AddExactlyOne([aulas_vars[(turma, disciplina, sala, horario)] for sala in salas for horario in horarios])

# Restrição 2: Um professor só pode dar uma aula de cada vez.
for professor in professores:
    for horario in horarios:
        aulas_do_professor_no_horario = []
        for turma in turmas:
            for disciplina in professor.disciplinas_que_leciona:
                if disciplina in turma.disciplinas:
                    for sala in salas:
                        aulas_do_professor_no_horario.append(aulas_vars[(turma, disciplina, sala, horario)])
        model.AddAtMostOne(aulas_do_professor_no_horario)

# Restrição 3: Uma turma só pode ter uma aula de cada vez.
for turma in turmas:
    for horario in horarios:
        model.AddAtMostOne([aulas_vars[(turma, disciplina, sala, horario)] for disciplina in turma.disciplinas for sala in salas])

# Restrição 4: Uma sala só pode ser usada por uma turma de cada vez.
for sala in salas:
    for horario in horarios:
        model.AddAtMostOne([aulas_vars[(turma, disciplina, sala, horario)] for turma in turmas for disciplina in turma.disciplinas])

# Restrição 5: Requisitos de sala (laboratório e capacidade).
for turma in turmas:
    for disciplina in turma.disciplinas:
        for sala in salas:
            for horario in horarios:
                var = aulas_vars[(turma, disciplina, sala, horario)]
                # Se a disciplina requer laboratório, a sala DEVE ser um laboratório.
                if disciplina.requer_laboratorio and not sala.eh_laboratorio:
                    model.Add(var == 0)
                # Se a disciplina NÃO requer laboratório, a sala NÃO DEVE ser um laboratório (opcional, mas bom para otimizar recursos).
                if not disciplina.requer_laboratorio and sala.eh_laboratorio:
                    model.Add(var == 0)
                # A capacidade da sala deve ser suficiente para o número de alunos.
                if sala.capacidade < turma.numero_de_alunos:
                    model.Add(var == 0)

print("Restrições adicionadas com sucesso!")

# --- Etapa 6: Resolução do Modelo ---

print("\n4. Resolvendo o modelo...")
solver = cp_model.CpSolver()
status = solver.Solve(model)

# --- Etapa 7: Exibição dos Resultados ---

if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
    print("Solução encontrada!\n")
    # Cria um dicionário para armazenar a grade para fácil impressão
    schedule = {sala.nome: {hr: "---" for hr in horarios} for sala in salas}
    
    for turma in turmas:
        for disciplina in turma.disciplinas:
            for sala in salas:
                for horario in horarios:
                    if solver.Value(aulas_vars[(turma, disciplina, sala, horario)]) == 1:
                        # Encontra o professor da disciplina
                        professor_da_aula = "N/A"
                        if disciplina.nome in professores_por_disciplina:
                            for p in professores_por_disciplina[disciplina.nome]:
                                professor_da_aula = p.nome
                                break
                        schedule[sala.nome][horario] = f"{turma.nome} - {disciplina.nome} ({professor_da_aula})"

    # Imprime a grade de horários no console
    header = ["Sala"] + horarios
    row_format = "{:<25}" + "{:<40}" * len(horarios)
    print(row_format.format(*header))
    print("-" * (25 + 40 * len(horarios)))
    for sala in salas:
        row_data = [sala.nome] + [schedule[sala.nome][hr] for hr in horarios]
        print(row_format.format(*row_data))

    # --- Etapa 8: Visualização Gráfica com Matplotlib ---
    fig, ax = plt.subplots(figsize=(18, 5))
    ax.axis('tight')
    ax.axis('off')
    
    cell_text = [[schedule[s.nome][h] for h in horarios] for s in salas]
    row_labels = [s.nome for s in salas]
    
    table = ax.table(cellText=cell_text, rowLabels=row_labels, colLabels=horarios, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(8)
    table.scale(1.2, 2)
    
    plt.title("Grade de Horários Gerada", fontsize=16)
    plt.show()

else:
    print("Não foi encontrada uma solução viável para o problema com as restrições atuais.")